# Imports

In [1]:
import json
from collections import Counter
from itertools import chain
from pathlib import Path

import numpy as np
import pandas as pd
import regex

# from unidecode import unidecode

In [60]:
from src.companies.processor import clean_company_type, normalize_company_name
from src.companies.utils import replace_company_types
from src.nif_validation.validation import get_nif_type, validate_nif
from src.utils.utils import fill_na, fill_to_length

# Load info

In [4]:
with open(r"C:\Users\josea\Downloads\genCat_Junio_2023.json", "r") as f:
    gencat = pd.json_normalize(json.load(f))

In [5]:
# df = pd.read_csv(r"C:\Users\josea\Downloads\Contractaci__p_blica_a_Catalunya__publicacions_a_la_Plataforma_de_serveis_de_contractaci__p_blica.csv")
df_emp = pd.read_csv(r"C:\Users\josea\Downloads\empresas.csv")

C:\Users\josea\AppData\Local\Temp\ipykernel_15848\1753786463.py:2: DtypeWarning: Columns (27,43,52,53,55,56,57,58,59,60,61,62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  df_emp = pd.read_csv(r"C:\Users\josea\Downloads\empresas.csv")


In [6]:
# Load data
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/insiders.parquet")
df_in = pd.read_parquet(dir_df)
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/outsiders.parquet")
df_ou = pd.read_parquet(dir_df)
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/minors.parquet")
df_mi = pd.read_parquet(dir_df)

In [7]:
df_in.columns = [".".join([el for el in c if el]) for c in df_in.columns]
df_ou.columns = [".".join([el for el in c if el]) for c in df_ou.columns]
df_mi.columns = [".".join([el for el in c if el]) for c in df_mi.columns]

# Aux functions
Functions necessary for processing the cells

In [8]:
def evaluate_cell(cell):
    if not isinstance(cell, (list, np.ndarray)):
        cell = [cell]
        # print("1", cell)
    if pd.isnull(cell[0]):
        return [None]
    elif isinstance(cell[0], str) and cell[0].startswith("[") and cell[0].endswith("]"):
        # print("2")
        return eval(cell[0])
    else:
        # print("3")
        return cell


def clean_df(df: pd.DataFrame):
    # Validate NIF
    df["ID"] = df["ID"].apply(validate_nif)
    # Clean company type
    df["Name"] = df["Name"].apply(clean_company_type, remove_type=False)
    # Remove company type
    df["Name_proc"] = df["Name"]
    # Normalize company name
    df["Name_norm"] = df["Name"].apply(normalize_company_name)

    return df


def nif_from_name(name):
    name_spl = np.array(name.split())
    valid = np.array([bool(validate_nif(s)) for s in name_spl])
    new_name = " ".join(name_spl[~valid])
    new_nif = Counter(name_spl[valid]).most_common()[0][0] if valid.any() else np.nan
    return new_name, new_nif

# Merge data from different sources

In [9]:
# def merge_data(
#     dir_metadata: Path,
#     merge_dfs=["minors", "insiders", "outsiders"],
# ):
#     """
#     Merge original data parquet files into single dataframe
#     """
#     dfs = [pd.read_parquet(dir_metadata.joinpath(f"{d}.parquet")) for d in merge_dfs]

#     # Unify texts from all sources
#     dfs_companies = []
#     for df in dfs:
#         # Reset index and rename to common identifier
#         index_names = df.index.names
#         orig_cols = df.columns
#         df.reset_index(inplace=True)
#         df["identifier"] = df[index_names].astype(str).agg("/".join, axis=1)
#         # df.drop(index_names, inplace=True, axis=1)
#         df.set_index("identifier", inplace=True)
#         df = df[orig_cols]

#         # Select company columns from winning parties and rename them
#         join_str = lambda x: ".".join([el for el in x if el])
#         joint_cnames = {join_str(c): c for c in df.columns}
#         reverse_joint_cnames = {v: k for k, v in joint_cnames.items()}
#         comp_cols = sorted([v for k, v in joint_cnames.items() if "WinningParty" in k])
#         print(comp_cols)

#         df_companies = df.loc[:, comp_cols]
#         use_cols = [reverse_joint_cnames[c].split(".")[-1] for c in comp_cols]
#         print(use_cols)
#         df_companies.columns = use_cols

#         dfs_companies.append(df_companies)
#         print()
#     df_companies = pd.concat(dfs_companies)

#     # Normalize info (lists of strings)
#     df_companies = df_companies.applymap(fill_na, fill=[None])
#     for c in df_companies.columns:
#         df_companies[c] = (
#             df_companies[c]
#             .apply(evaluate_cell)
#             .apply(
#                 lambda x: [None] if not x[0] else [str(el).strip().lower() for el in x]
#             )
#         )
#     return df_companies

# df_companies = merge_data(
#     dir_metadata=Path("C:/Users/josea/Documents/Trabajo/data/metadata/")
# )
# df_companies.to_parquet("companies.parquet")

In [ ]:
df_companies = pd.read_parquet("companies.parquet")

# Obtain individual companies

In [17]:
# Use only those where all dimensions match
# (e.g. same number of companies and companies ids)
# and drop NAs
df_companies = df_companies[
    df_companies[["ID", "Name"]]
    .applymap(lambda x: not pd.isna(x[0]))
    .apply(all, axis=1)
]
df_companies = df_companies[
    df_companies.applymap(lambda x: len(x) if x[0] else None).apply(
        lambda x: len(set([el for el in x if not pd.isnull(el)])) == 1,
        axis=1,
    )
]
companies_columns = list(df_companies.columns)
# Get number of companies by tender
df_companies["_len"] = df_companies["ID"].apply(len)

# Fill lists of None to have the same number of elements and explode later
companies = pd.DataFrame(
    df_companies.apply(
        lambda x: [fill_to_length(list(el), x[-1]) for el in x[:-1]], axis=1
    ).tolist(),
    columns=companies_columns,
)

# Split companies in rows
companies = companies.explode(companies_columns)

In [18]:
companies.head()

,ID,CompanyTypeCode,Name,CityName,IdentificationCode,PostalZone,CountrySubentityCode
0,b30437347,None,climayor s.l. b30437347,None,None,None,None
1,b60564309,None,"gometrics, s.l.",None,None,None,None
2,g57694549,None,associaciò alcem el c.i.n.e.,None,None,None,None
3,b73326019,None,diseño y decoraciones j. peñalver s.l. b73326019,None,None,None,None
4,b28954170,None,"thermo fisher scientific, s.l.",None,None,None,None


In [40]:
companies

,ID,CompanyTypeCode,Name,CityName,IdentificationCode,PostalZone,CountrySubentityCode,Name_proc,Name_norm
0,b30437347,None,climayor s.l. b30437347,None,None,None,None,climayor s.l. b30437347,climayorslb30437347
1,b60564309,None,gometrics s.l.,None,None,None,None,gometrics s.l.,gometricssl
2,g57694549,None,associaciò alcem el c.i.n.e.,None,None,None,None,associaciò alcem el c.i.n.e.,associacioalcemelcine
3,b73326019,None,diseño y decoraciones j. peñalver s.l. b73326019,None,None,None,None,diseño y decoraciones j. peñalver s.l. b73326019,disenoydecoracionesjpenalverslb73326019
4,b28954170,None,thermo fisher scientific s.l.,None,None,None,None,thermo fisher scientific s.l.,thermofisherscientificsl
...,...,...,...,...,...,...,...,...,...
2260787,r1100131j,None,hospital san juan grande,None,None,None,None,hospital san juan grande,hospitalsanjuangrande
2260787,b90368192,None,integracion terapeutica diogenes s.l.p.,None,None,None,None,integracion terapeutica diogenes s.l.p.,integracionterapeuticadiogenesslp
2260787,g11299450,None,asoc vejeriega ayuda discapacitado,None,None,None,None,asoc vejeriega ayuda discapacitado,asocvejeriegaayudadiscapacitado
2260787,g11014909,None,asociación para la atención de personas con ne...,None,None,None,None,asociación para la atención de personas con ne...,asociacionparalaatenciondepersonasconnecesidad...


In [24]:
companies2 = clean_df(companies)
# Aggregate company info in lists
companies2 = (
    companies2
    # companies[["ID", "Name", "Name_proc", "Name_norm"]]
    .groupby(["ID", "Name_norm"])
    .agg(list)
    .reset_index()
)
companies2["count"] = companies2["Name_proc"].apply(len)

In [25]:
# Choose definitive values
def suggest_value(elements):
    """
    Select elements based on appearance.
    If same number of appearances, choose the longest.
    If shorter elements are not included in the 'main' one, return all.
    """
    cnt = Counter(elements)
    cnt.pop(None, None)
    cnt = cnt.most_common()
    if cnt:
        max_cnt = cnt[0][1]
        els = sorted([k for k, v in cnt if v == max_cnt], key=lambda x: (-len(x), x))
        # return els[0]
        base = els.pop(0)
        return [base]
        # if all(
        #     [all(t in base for t in regex.sub(r"\W", " ", el).split()) for el in els]
        # ):
        #     return [base]
        # return [base] + els
    else:
        # return None
        return [None]

#### Unique names and IDs

In [26]:
# Unique names and IDs
# These companies have always appeared with the same (id-name) association
cols_vals = [c for c in companies2.columns if c not in ["ID", "Name_norm", "count"]]
unique_ID = ~companies2["ID"].duplicated(keep=False)
unique_NAME = ~companies2["Name_norm"].duplicated(keep=False)
unique = companies2[unique_ID & unique_NAME].reset_index()

non_unique_ids = list(set(companies2.index) - set(unique["index"]))
non_unique = companies2.loc[non_unique_ids]

unique["index"] = unique["index"].apply(lambda x: [x])

In [27]:
unique.shape

(213414, 11)

#### Repeated IDs and Names

In [28]:
# Repeated IDs
def unify_repeated_col(df: pd.DataFrame, rep_col: str, un_col: str):
    """
    Parameters
    ----------
    df: pd.DataFrame
    rep_col: str
        Name of column with repeated values that will be unified
    un_col: str
        Name of column with non unique values
    """
    # Non-unique columns
    cols_vals = [c for c in df.columns if c not in [rep_col, "count"]]
    repeated_rows = df[rep_col].duplicated(keep=False)
    repeated = df[repeated_rows]

    # Count times the values appear
    repeated.loc[repeated.index, [un_col]] = (
        repeated.loc[repeated.index, un_col].apply(lambda x: [x])
        * repeated.loc[repeated.index, "count"]
    )
    # Group by repeated
    repeated = repeated.reset_index()
    repeated = repeated.groupby(rep_col).agg(
        {
            "index": list,
            **{c: lambda x: list(chain.from_iterable(x)) for c in cols_vals},
            "count": sum,
        }
    )
    # Get the most common values for each column
    repeated.loc[repeated.index, un_col] = (
        repeated.loc[repeated.index, un_col].apply(suggest_value).values
    )
    repeated = repeated.reset_index()

    # Concatenate unique
    use_index = repeated.loc[repeated[un_col].apply(len) == 1, un_col].index
    repeated.loc[use_index, un_col] = repeated.loc[use_index, un_col].apply(
        lambda x: x[0]
    )
    unified = repeated.loc[use_index]

    return unified

In [29]:
unified_ID = unify_repeated_col(non_unique, "ID", "Name_norm")
# Update non_unique
non_unique_ids = list(
    set(non_unique.index) - set(chain.from_iterable(unified_ID["index"]))
)
non_unique = companies2.loc[non_unique_ids]

unified_NAME = unify_repeated_col(non_unique, "Name_norm", "ID")
# Update non_unique
non_unique_ids = list(
    set(non_unique.index) - set(chain.from_iterable(unified_NAME["index"]))
)
non_unique = companies2.loc[non_unique_ids]

# Get the rest
non_unique = non_unique.reset_index()
non_unique["index"] = non_unique["index"].apply(lambda x: [x])

#### Companies info

In [30]:
# Global
merged_global = pd.concat([unique, unified_ID, unified_NAME, non_unique])
cols_vals = [c for c in companies.columns if c not in ["ID", "Name_norm", "count"]]
merged_global = merged_global.groupby(["ID", "Name_norm"]).agg(
    {
        "index": lambda x: list(chain.from_iterable(x)),
        **{c: lambda x: list(chain.from_iterable(x)) for c in cols_vals},
        "count": sum,
    }
)
merged_global = merged_global.reset_index()


unified_ID2 = unify_repeated_col(merged_global, "ID", "Name_norm")
merged_global = merged_global[~merged_global["ID"].isin(unified_ID2["ID"])]
# non_unique = companies.loc[non_unique_ids]

unified_NAME2 = unify_repeated_col(merged_global, "Name_norm", "ID")
merged_global = merged_global[
    ~merged_global["Name_norm"].isin(unified_NAME2["Name_norm"])
]
# # Update non_unique
# non_unique_ids = list(set(non_unique.index) - set(chain.from_iterable(unified_NAME["index"])))
# non_unique = companies.loc[non_unique_ids]

final_merged = pd.concat(
    [
        merged_global,
        unified_ID2,
        unified_NAME2,
    ]
).reset_index(drop=True)

final_merged.loc[final_merged.index, cols_vals] = (
    final_merged.loc[final_merged.index, cols_vals]
    .applymap(suggest_value)
    .applymap(lambda x: x[0])
)

In [31]:
final_merged[["Name_2", "ID_2"]] = final_merged["Name"].apply(nif_from_name).tolist()

In [39]:
clean_company_type("sacyr neopul s.a s.e.e.")

'sacyr neopul s.a. s.e.e.'

In [44]:
companies.loc[companies["ID"]=="d28337145"]

,ID,CompanyTypeCode,Name,CityName,IdentificationCode,PostalZone,CountrySubentityCode,Name_proc,Name_norm
63992,d28337145,None,vorwerk españa management s.l s.e.n.c.,None,None,None,None,vorwerk españa management s.l. s.e.n.c.,vorwerkespanamanagementslsenc
125837,d28337145,None,vorwerk espa\a m.s.l s.e.n.c.,None,None,None,None,vorwerk espa\a m.s.l s.e.n.c.,vorwerkespaamslsenc
364020,d28337145,None,vorwerk españa m.s.l.s.c.,None,None,None,None,vorwerk españa m.s.l.s.c.,vorwerkespanamslsc
493308,d28337145,None,"vorwek españa, m.s.l s.e.n.c.",None,None,None,None,"vorwek españa, m.s.l s.e.n.c.",vorwekespanamslsenc
670035,d28337145,None,vorwerk espa\a m.s.l s.e.n.c.,None,None,None,None,vorwerk espa\a m.s.l s.e.n.c.,vorwerkespaamslsenc
707014,d28337145,None,vorwerk españa m.s.l,None,None,None,None,vorwerk españa m.s.l,vorwerkespanamsl
878481,d28337145,None,vorwerk españa management s.l s.e.n.c.,None,None,None,None,vorwerk españa management s.l. s.e.n.c.,vorwerkespanamanagementslsenc
971844,d28337145,None,vorwerk españa m.s.l s.e.n.c.,None,None,None,None,vorwerk españa m.s.l s.e.n.c.,vorwerkespanamslsenc
1013278,d28337145,None,vorwerk españa management s.l s.e.n.c.,None,None,None,None,vorwerk españa management s.l. s.e.n.c.,vorwerkespanamanagementslsenc
1266250,d28337145,None,vorwerk españa m.s.l s.e.n.c.,None,None,None,None,vorwerk españa m.s.l s.e.n.c.,vorwerkespanamslsenc


In [45]:
# def clean_df2(df: pd.DataFrame):
#     # Validate NIF
#     df["ID"] = df["ID"].apply(validate_nif)
#     # Clean company type
#     df["Name"] = df["Name"].apply(clean_company_type, remove_type=False)
#     # Remove company type
#     df["Name_proc"] = df["Name"]
#     # Normalize company name
#     df["Name_norm"] = df["Name"].apply(normalize_company_name)

#     return df
# clean_df2(companies.loc[companies["ID"]=="d28337145"])

In [33]:
final_merged

,ID,Name_norm,index,CompanyTypeCode,Name,CityName,IdentificationCode,PostalZone,CountrySubentityCode,Name_proc,count,Name_2,ID_2
0,00000000t,singaporeexhibitionservices,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",None,singapore exhibition services,None,es,None,None,singapore exhibition services,46,singapore exhibition services,NaN
1,00021492x,carmenbalgueriasjimenez,[42],None,carmen balguerias jiménez,None,None,None,None,carmen balguerias jiménez,2,carmen balguerias jiménez,NaN
2,00024509z,uteondoangiroatxagorritxu,[43],None,u.t.e. ondoan giroa txagorritxu,None,None,None,None,u.t.e. ondoan giroa txagorritxu,1,u.t.e. ondoan giroa txagorritxu,NaN
3,00035211k,palomasainzdelamazadelaserna,[44],None,paloma sáinz de la maza de la serna,None,None,None,None,paloma sáinz de la maza de la serna,1,paloma sáinz de la maza de la serna,NaN
4,00067665e,albertodelgadocebrian,[45],None,alberto delgado cebrián,None,None,None,None,alberto delgado cebrián,1,alberto delgado cebrián,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
288885,b28672764,xylemwatersolutionsespanaslu,"[146213, 146214, 229008, 229009, 229010, 22901...",None,xylem water solutions españa s.l.u.,madrid,es,28022.0,es300,xylem water solutions españa s.l.u.,134,xylem water solutions españa s.l.u.,NaN
288886,a28011153,zardoyaotissa,"[56871, 142513, 142514, 142515, 142516, 142517...",None,zardoya otis s.a.,madrid,es,28033.0,es300,zardoya otis s.a.,1638,zardoya otis s.a.,NaN
288887,b83230870,zesaurotraduccionessl,"[74653, 74654, 331632, 331633]",None,zesauro traducciones s.l.,None,es,None,None,zesauro traducciones s.l.,7,zesauro traducciones s.l.,NaN
288888,b29060381,zimmerbiometspainslu,"[147594, 147595, 147596, 229490, 229491, 22949...",None,zimmer biomet spain s.l.u.,barcelona,es,8038.0,es511,zimmer biomet spain s.l.u.,574,zimmer biomet spain s.l.u.,NaN


In [ ]:
final_merged[final_merged["ID"] == "b30437347"]

,ID,Name_norm,index,CompanyTypeCode,Name,CityName,IdentificationCode,PostalZone,CountrySubentityCode,count,Name_2,ID_2
478,b30437347,climayorslb30437347,[491],None,climayor s.l. b30437347,None,None,None,None,1,climayor s.l.,b30437347


In [46]:
# Find UTEs based on name
ute_n = (
    final_merged[["Name", "Name_proc"]]
    .apply(lambda x: regex.search(r"u.t.e.", " ".join(x)), axis=1)
    .apply(bool)
)
# Find UTEs based on ID
ute_i = final_merged["ID"].apply(lambda x: x.startswith("u"))

# final_merged[ute_i | ute_n]["Name"].values
# sum(ute_n), sum(ute_i), sum(ute_n & ute_i), sum(ute_n & ute_i)/min(sum(ute_n), sum(ute_i))

In [90]:
final_merged[ute_i].head()

,ID,Name_norm,index,CompanyTypeCode,Name,CityName,IdentificationCode,PostalZone,CountrySubentityCode,Name_proc,count,Name_2,ID_2
277369,u01516699,utealegriasarasola,[426734],None,u.t.e. alegría sarasola,None,None,None,None,u.t.e. alegría sarasola,1,u.t.e. alegría sarasola,NaN
277370,u01523190,utealegriasarasola2,[426735],None,u.t.e. alegria-sarasola 2,None,None,None,None,u.t.e. alegria-sarasola 2,1,u.t.e. alegria-sarasola 2,NaN
277371,u01548726,uteindenortpvslproviseribericasluteieslizardi,[426736],None,u.t.e. indenort pv s.l. -proviser iberica s.l....,None,None,None,None,u.t.e. indenort pv s.l. -proviser iberica s.l....,1,u.t.e. indenort pv s.l. -proviser iberica s.l....,NaN
277372,u01560382,utealegriasarasola4,[426737],None,u.t.e. alegria sarasola 4,None,None,None,None,u.t.e. alegria sarasola 4,14,u.t.e. alegria sarasola 4,NaN
277373,u01561570,utetranviauniversidad,[426738],None,u.t.e. tranvía universidad,None,None,None,None,u.t.e. tranvía universidad,1,u.t.e. tranvía universidad,NaN


## Empresas Zaragoza

In [61]:
# df = pd.read_csv(r"C:\Users\josea\Downloads\empresas.csv", sep=";", header=0, nrows=64, index_col=False)
# df = pd.read_excel(r"C:\Users\josea\Downloads\empresas.xlsx")

with open(r"C:\Users\josea\Downloads\empresas_zgz.csv", "r", encoding="utf-8") as f:
    emp = [
        [el.replace('"', "").strip() for el in l.lower().strip().split(";", 4)]
        for l in f.readlines()
        if len(l) > 2
    ]
cols = emp[0]
data = emp[1:]
emp_zgz = pd.DataFrame(data=data, columns=cols)
emp_zgz = emp_zgz.applymap(lambda x: x if x else None)
emp_zgz = emp_zgz.dropna().drop_duplicates().reset_index(drop=True)
emp_zgz["empresa"] = emp_zgz["empresa"].apply(replace_company_types)
emp_zgz["nif"] = emp_zgz["nif"].apply(lambda x: regex.sub(r"\W", "", x))
emp_zgz["nif_type"] = emp_zgz["nif"].apply(get_nif_type)

In [62]:
emp_zgz.head()

,nombre,expediente,empresa,nif,texto,nif_type
0,alquiler de un equipo de iluminación para esce...,cul09-19,fuse records-fluge zaragoza s.l.,b99053506,https://contrataciondelestado.es/wps/poc?uri=d...,CIF
1,derribo de la estructura actual de polipasto y...,eco2019335-19,depuracion de aguas de mediterraneo s.l.,b96456553,https://contrataciondelestado.es/wps/poc?uri=d...,CIF
2,obras de acondicionamiento de la planta 3ª de ...,viv01-19,inardec construccion creativa s.l.,b99413874,https://contrataciondelestado.es/wps/poc?uri=d...,CIF
3,obras de acondicionamiento de la planta 3ª de ...,viv01-19,construcciones rubio morte s.a.,a50070663,https://contrataciondelestado.es/wps/poc?uri=d...,CIF
4,servicio para la realización del programa depo...,dep01-2019,serveo servicios s.a.u. antigua (ferrovial ser...,a80241789,https://contrataciondelestado.es/wps/poc?uri=d...,CIF


## Compare companies from Tenders and Zaragoza

In [71]:
# Same nif
common_nif = set(final_merged["ID"]) & set(emp_zgz["nif"])
common_comp_nif = final_merged[final_merged["ID"].isin(common_nif)][
    ["ID", "Name", "count"]
].drop_duplicates()
common_emp_nif = emp_zgz[emp_zgz["nif"].isin(common_nif)][
    ["nif", "empresa"]
].drop_duplicates()
c_nif = pd.merge(
    common_comp_nif, common_emp_nif, left_on="ID", right_on="nif"
).reset_index(drop=True)
c_nif.head()

,ID,Name,count,nif,empresa
0,07046603r,alba mª ortega altamirano,1,07046603r,alba m.ª ortega altamirano
1,07823314w,jesus gutierrez sierra,225,07823314w,jesús gutiérrez sierra
2,17144195h,jaime macipe gayarre,8,17144195h,jaime macipe gayarre
3,17215345y,pascual pardos sierra,1,17215345y,pascual pardos sierra
4,17747571n,dª maría nila atienza fanlo,1,17747571n,mª nila atienza fanlo


In [72]:
# Same name
common_name = set(final_merged["Name"]) & set(emp_zgz["empresa"])
common_comp_name = final_merged[final_merged["Name"].isin(common_name)][
    ["ID", "Name", "count"]
].drop_duplicates()
common_emp_name = emp_zgz[emp_zgz["empresa"].isin(common_name)][
    ["nif", "empresa"]
].drop_duplicates()
c_name = pd.merge(
    common_comp_name, common_emp_name, left_on="Name", right_on="empresa"
).reset_index(drop=True)
c_name.head()

,ID,Name,count,nif,empresa
0,17144195h,jaime macipe gayarre,8,17144195h,jaime macipe gayarre
1,17215345y,pascual pardos sierra,1,17215345y,pascual pardos sierra
2,25138444l,susana del río sanz,1,25138444l,susana del río sanz
3,25445647b,leonardo oro vargas,2,25445647b,leonardo oro vargas
4,25449076j,ana belen gines hidalgo,6,25449076j,ana belen gines hidalgo


In [81]:
common_comp_name

,ID,Name,count
22412,17144195h,jaime macipe gayarre,8
22558,17215345y,pascual pardos sierra,1
34926,25138444l,susana del río sanz,1
35688,25445647b,leonardo oro vargas,2
35702,25449076j,ana belen gines hidalgo,6
...,...,...,...
288683,a79252219,securitas seguridad españa s.a.,877
288702,a25027145,servicios microinformatica s.a.,1610
288744,a08602815,talher s.a.,243
288797,a81356313,unitronics comunicaciones s.a.,162


In [83]:
# Common name, different nif
c_name_diff_nif_comp_ids = list(set(common_comp_name.index) - set(common_comp_nif.index))
c_name_diff_nif_emp_ids = list(set(common_emp_name.index) - set(common_emp_nif.index))

In [84]:
emp_zgz.loc[c_name_diff_nif_emp_ids].head()

,nombre,expediente,empresa,nif,texto,nif_type
450,suministro e instalación de cuatro pantallas d...,0021483-21,imagic vision s.l.,b840444205,https://contrataciondelestado.es/wps/poc?uri=d...,None
387,servicio de mantenimiento del diseño gráfico d...,0020126-21,ht publicidad grupo tafalla s.l.,g50493097,https://contrataciondelestado.es/wps/poc?uri=d...,CIF
39,explotación del servicio de bares y hostelería...,dep08-19,beberapid c.b.,e98134752,https://contrataciondelestado.es/wps/poc?uri=d...,CIF
552,contratación de los servicios de bar-cafetería...,0076573-21,ansamater s.l.,bb09671256,https://contrataciondelestado.es/wps/poc?uri=d...,None
16,"suministro de reactivos, material fungible, me...",0529565-19,sanilabo s.l.,a96141361,https://contrataciondelestado.es/wps/poc?uri=d...,CIF


In [85]:
final_merged.loc[c_name_diff_nif_comp_ids].head()

,ID,Name_norm,index,CompanyTypeCode,Name,CityName,IdentificationCode,PostalZone,CountrySubentityCode,Name_proc,count,Name_2,ID_2
244672,b98134752,beberapidcb,[374767],None,beberapid c.b.,None,None,None,None,beberapid c.b.,1,beberapid c.b.,NaN
131652,b09671256,ansamatersl,[190537],None,ansamater s.l.,None,None,None,None,ansamater s.l.,1,ansamater s.l.,NaN
200903,b67375006,xiorstudenthousingspainslu,[302949],None,xior student housing spain s.l.u.,None,None,None,None,xior student housing spain s.l.u.,1,xior student housing spain s.l.u.,NaN
240488,b96141361,sanilabosl,"[367320, 367321, 367322]",None,sanilabo s.l.,None,es,None,None,sanilabo s.l.,121,sanilabo s.l.,NaN
184750,b50941855,agenciaaragonesadenoticiassl,[275878],None,agencia aragonesa de noticias s.l.,None,None,None,None,agencia aragonesa de noticias s.l.,10,agencia aragonesa de noticias s.l.,NaN


In [91]:
# c_concat = pd.concat([c_nif, c_name])
# c_concat[~c_concat.duplicated(keep=False)]

In [87]:
set(emp_zgz["nif"])-set(final_merged["ID"])

{'17410114f',
 '17732626v',
 '17763291t',
 '18426455m',
 '514863471',
 '5412400v',
 '7992498k',
 'a27178789b50113562',
 'a50004431',
 'a50016666',
 'a50032002',
 'a50169333',
 'a81196743',
 'a96141361',
 'b01644889',
 'b06851471',
 'b50349323',
 'b50709641',
 'b50755149',
 'b50931302',
 'b54941855',
 'b673750006',
 'b79309902',
 'b82381682',
 'b840444205',
 'b84049238',
 'b85508059',
 'b9691037',
 'b99399438',
 'b99412629',
 'b99532574',
 'bb09671256',
 'e98134752',
 'g50493097',
 'n0171609a',
 'q99118598',
 'siret56211521200077',
 'sl',
 'u02778884',
 'u99489411'}

In [ ]:
common_nif = set(df["nif"]) - set(final_merged["ID"])
common_comp = final_merged[final_merged["ID"].isin(common_nif)][
    ["ID", "Name"]
].drop_duplicates()
common_emp = df[df["nif"].isin(common_nif)][["nif", "empresa"]].drop_duplicates()

In [ ]:
# dup_emp = df["empresa"].duplicated(keep=False)
# dup_nif = df["nif"].duplicated(keep=False)